In [63]:
import sagemaker
from sagemaker.pytorch import PyTorch
import boto3
import json

In [64]:
sagemaker_session = sagemaker.Session()
bucket = 'slip-ml'
role = 'arn:aws:iam::438465160412:role/Sagemaker'
project_name = 'vid-embed-image-transformer'

In [65]:
secret_name = "huggingface"
region_name = "us-east-1"
session = boto3.session.Session()
secretsmanager = session.client(service_name='secretsmanager', region_name=region_name)
get_secret_value_response = secretsmanager.get_secret_value(SecretId=secret_name)
secret = get_secret_value_response['SecretString']
api_key = json.loads(secret)["API_KEY"]

[05/29/25 14:22:55] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=157870;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=458933;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/botocore/credentials.py#1352\1352]8;;\

In [66]:
training_instances_gpus = {
    "ml.g5.2xlarge": 1,
    "ml.g5.12xlarge": 4,
    "ml.p4d.24xlarge": 8,
    "ml.p5.48xlarge": 8
}
instance_type = 'ml.p4d.24xlarge'

In [67]:
image_uri = sagemaker.image_uris.retrieve(framework='pytorch',
                             region=sagemaker_session.boto_region_name,
                             instance_type=instance_type,
                             image_scope='training')
print(image_uri)

                    INFO     Defaulting to only available Python version: py312                   ]8;id=846015;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=247776;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/image_uris.py#610\610]8;;\

763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.6.0-gpu-py312


In [ ]:
estimator = PyTorch(
    entry_point="vallr.py",
    role=role,
    base_job_name=project_name,
    instance_count=1,
    instance_type=instance_type,
    image_uri=image_uri,
    py_version="py310",
    source_dir="source",
    hyperparameters={
        "batch-size": 2,
        "epochs": 7,
        "lr": 3e-4,
        "project-name": f"{project_name}",
    },
    sagemaker_session=sagemaker_session,
    volume_size=50,
    environment={"HF_TOKEN": "" + api_key},
    output_path=f's3://{bucket}/models/{project_name}',
    code_location=f's3://{bucket}/model-building/{project_name}',
)

In [ ]:
estimator.fit({'training': f's3://{bucket}/data/vallr/train/',
               'test': f's3://{bucket}/data/vallr/test/'})

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=467754;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=134486;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[05/29/25 14:22:56] INFO     Creating training-job with name:                                       ]8;id=237056;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=190223;file:///Users/emmettstorts/.local/share/virtualenvs/slip-ml-bXUTykFe/lib/python3.11/site-packages/sagemaker/session.py#1042\1042]8;;\
                             vid-embed-image-transformer-2025-05-29-20-22-55-923                                   

2025-05-29 20:22:57 Starting - Starting the training job
2025-05-29 20:22:57 Pending - Training job waiting for capacity..